# Make some CMR tables for ABoVE

This notebook queries CMR for collections and granules for ABoVE (though all functions are generic and will work with any query parameters if you familiarize yourself with arguments) and are pickled to binary files for use with the other jupyter notebook.


## Imports and files

In [1]:
import json
import requests
import pandas as pd

from shapely.geometry import shape, mapping, box
from shapely.ops import cascaded_union

# path to above-stm
repo = "/home/jack/Desktop/git/above-stm/"

## CMR Requests

In [2]:
def cmr_search(endpoint, parameters, update=None):
    """ """
    
    cmr_collections = "https://cmr.earthdata.nasa.gov/search/"
    
    # join query parameters and there values 
    query = "&".join([param+"="+value for param, value in parameters.items()])
    
    # combine with the cmr search endpoint and submit get request
    response = requests.get(cmr_collections+endpoint+query)
    
    # parse json response to dictionary
    data = json.loads(response.text)
    
    # if a path is given, update local copy
    if update:
        try:
            with open(update, "w") as f:
                f.write(json.dumps(data, indent=4))
        except:
            print("Could not write response to json. Skipping.")
    
    return(data)

### Collections
#### Request new ABoVE datasets from CMR

In [3]:
cmr_params = dict(project="ABoVE", page_size="1000")
above_datasets = cmr_search(
    "collections.umm_json_v1_4?", 
    cmr_params, 
    update=repo+"data/2019-05-11_above_datasets.json")

above_datasets.keys()

dict_keys(['hits', 'took', 'items'])

#### Read to dict from a local file

In [4]:
with open(repo+"data/2019-05-11_above_datasets.json", "r") as f:
    above_datasets = json.load(f)

### Granules
#### Request new ABoVE granules from CMR

In [5]:
cmr_params = dict(collection_concept_id="C1456630397-ORNL_DAAC", page_size="2000")
above_dataset_granules = cmr_search("granules.umm_json_v1_4?", cmr_params)

above_dataset_granules.keys()

dict_keys(['hits', 'took', 'items'])

# Parsing CMR Data

## CMR Collections
#### Functions to help populate datasets table

In [6]:
def get_bounds(umm):
    
    umm_extent = umm["SpatialExtent"]
    umm_geometry = umm_extent["HorizontalSpatialDomain"]["Geometry"]
    umm_bounds = umm_geometry["BoundingRectangles"][0]

    minx = umm_bounds["WestBoundingCoordinate"]
    maxy = umm_bounds["NorthBoundingCoordinate"]
    maxx = umm_bounds["EastBoundingCoordinate"]
    miny = umm_bounds["SouthBoundingCoordinate"]

    return([minx, miny, maxx, maxy])


def get_bounds_shapely(bounds):
    """ """
    
    shapely_box = box(
       float(bounds[0]),    # minx 
       float(bounds[1]),    # miny
       float(bounds[2]),    # maxx
       float(bounds[3]))    # maxy
    
    return(shapely_box)


def get_science_keywords(umm):
    """ """
    
    science_keywords = umm["ScienceKeywords"]
    keywords = []
    
    for keyset in science_keywords:
        for keyword, value in keyset.items():
            if value not in keywords:
                keywords.append(value)
        
    return(keywords)


def get_urls(umm, search):
    """ """
    
    if search=="datasets":
        
        url_landingpage = None
        url_documentation = None
        url_datapool = None
        url_sdat = None
        url_thredds = None

        for RelatedUrls in umm["RelatedUrls"]:

            try:

                if RelatedUrls["Relation"][0]=="DATA SET LANDING PAGE":
                    url_landingpage = RelatedUrls["URLs"][0]

                elif RelatedUrls["Description"]=="ORNL DAAC Data Set Documentation":
                    url_documentation = RelatedUrls["URLs"][0]

                elif RelatedUrls["Relation"][0]=="GET DATA":
                    url_datapool = RelatedUrls["URLs"][0]

                else:

                    for url in RelatedUrls["URLs"]:

                        if "webmap" in url:
                            url_sdat = url

                        elif "thredds" in url:
                            url_thredds = url

                        else:
                            pass
            except:

                pass

        return((url_landingpage,
                url_documentation,
                url_datapool,
                url_sdat,
                url_thredds))
    
    elif search=="granules":
        
        url_datapool = None
        
        for RelatedUrls in umm["RelatedUrls"]:
            try:
                if RelatedUrls["Type"]=="GET DATA":
                    url_datapool = RelatedUrls["URL"]
            except:
                pass

        return(url_datapool)
    
    else:
        
        return(None)


def get_granule_parameters(umm):
    """ """
    
    granule_parameters = []
    for param in umm["MeasuredParameters"]:
        for name, value in param.items():
            if value not in granule_parameters:
                granule_parameters.append(value)
                
    return(granule_parameters)

#### Function to parse CMR collections query response to a table of datasets:

In [7]:
def get_datasets_table(cmr_collections_response_dictionary):
    """ """
    
    dataset_rows = []

    for dataset in cmr_collections_response_dictionary["items"]:
        
        umm = dataset["umm"]
        meta = dataset["meta"]
        archive = meta["provider-id"]
        conceptid = meta["concept-id"]

        if len(umm["Projects"])==1:
        
            # PUBLICATION
            short_name = umm["ShortName"]
            title = umm["EntryTitle"]
            science_keywords = get_science_keywords(umm)
            #citation = umm["CollectionCitations"][0]["OtherCitationDetails"]
            #doi = umm["CollectionCitations"][0]["DOI"]["DOI"]
            progress = umm["CollectionProgress"]

            # URLS
            urls = get_urls(umm, "datasets")
            url_landingpage = urls[0]
            url_documentation = urls[1]
            url_datapool = urls[2]
            url_sdat = urls[3]
            url_thredds = urls[4]

            # TEMPORAL
            range_time = umm["TemporalExtents"][0]["RangeDateTimes"]
            start_time = range_time[0]["BeginningDateTime"]
            end_time = range_time[0]["EndingDateTime"]

            # SPATIAL
            try:
                bounds = get_bounds(umm)
                bounds_shapely = get_bounds_shapely(bounds)
                minlon = bounds[0]
                minlat = bounds[1]
                maxlon = bounds[2]
                maxlat = bounds[3]

                dataset_rows.append((
                    title,
                    archive,
                    conceptid,
                    short_name,
                    start_time,
                    end_time,
                    minlon,
                    minlat,
                    maxlon,
                    maxlat,
                    #doi,
                    #citation,
                    progress,
                    science_keywords,
                    url_landingpage,
                    url_documentation,
                    url_datapool,
                    url_sdat,
                    url_thredds,
                    bounds_shapely
                ))
            except:
                pass


    return(pd.DataFrame(dataset_rows, columns=[
        "title",
        "archive",
        "conceptid",
        "short_name",
        "start_time",
        "end_time",
        "minlon",
        "minlat",
        "maxlon",
        "maxlat",
        #"doi",
        #"citation",
        "progress",
        "science_keywords",
        "url_landingpage",
        "url_documentation",
        "url_datapool",
        "url_sdat",
        "url_thredds",
        "bounds_shapely"
    ]))

#### Get datasets table (function above):

In [8]:
dataset_table = get_datasets_table(above_datasets)

dataset_table

,title,archive,conceptid,short_name,start_time,end_time,minlon,minlat,maxlon,maxlat,progress,science_keywords,url_landingpage,url_documentation,url_datapool,url_sdat,url_thredds,bounds_shapely
0,A Concise Experiment Plan for the Arctic-Borea...,ORNL_DAAC,C1536961538-ORNL_DAAC,ABoVE_Concise_Experiment_Plan_1617,2014-01-01T00:00:00.000Z,2021-12-31T23:59:59.000Z,-176.12,39.42,-66.92,81.61,COMPLETE,"[EARTH SCIENCE, ATMOSPHERE, ATMOSPHERIC CHEMIS...",https://doi.org/10.3334/ORNLDAAC/1617,https://daac.ornl.gov/ABOVE/guides/ABoVE_Conci...,https://daac.ornl.gov/daacdata/above/ABoVE_Con...,None,None,"POLYGON ((-66.92 39.42, -66.92 81.61, -176.12 ..."
1,ABoVE LVIS L1B Geolocated Return Energy Wavefo...,NSIDC_ECS,C1513105920-NSIDC_ECS,ABLVIS1B,2017-06-29T00:00:00.000Z,2017-07-17T23:59:59.999Z,-158.00,48.00,-104.00,72.00,PLANNED,"[EARTH SCIENCE, SPECTRAL/ENGINEERING, INFRARED...",https://doi.org/10.5067/UMRAWS57QAFU,None,https://search.earthdata.nasa.gov/search?q=ABL...,None,None,"POLYGON ((-104 48, -104 72, -158 72, -158 48, ..."
2,ABoVE LVIS L2 Geolocated Surface Elevation Pro...,NSIDC_ECS,C1513105984-NSIDC_ECS,ABLVIS2,2017-06-29T00:00:00.000Z,2017-07-17T23:59:59.999Z,-158.00,48.00,-104.00,72.00,PLANNED,"[EARTH SCIENCE, LAND SURFACE, TOPOGRAPHY, TERR...",https://doi.org/10.5067/IA5WAX7K3YGY,None,https://search.earthdata.nasa.gov/search?q=ABL...,None,None,"POLYGON ((-104 48, -104 72, -158 72, -158 48, ..."
3,ABoVE: Active Layer and Soil Moisture Properti...,ORNL_DAAC,C1608833566-ORNL_DAAC,ABoVE_PBand_SAR_1657,2014-08-16T00:00:00.000Z,2017-10-10T23:59:59.000Z,-167.94,64.71,-150.25,70.88,COMPLETE,"[EARTH SCIENCE, LAND SURFACE, FROZEN GROUND, A...",https://doi.org/10.3334/ORNLDAAC/1657,https://daac.ornl.gov/ABOVE/guides/ABoVE_PBand...,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,None,None,"POLYGON ((-150.25 64.70999999999999, -150.25 7..."
4,ABoVE: AirSWOT Color-Infrared Imagery Over Ala...,ORNL_DAAC,C1574863711-ORNL_DAAC,ABoVE_Open_Water_Map_1643,2017-07-09T00:00:00.000Z,2017-08-17T23:59:59.000Z,-149.25,46.85,-98.64,69.47,COMPLETE,"[EARTH SCIENCE, SPECTRAL/ENGINEERING, INFRARED...",https://doi.org/10.3334/ORNLDAAC/1643,https://daac.ornl.gov/ABOVE/guides/ABoVE_Open_...,https://daac.ornl.gov/daacdata/above/ABoVE_Ope...,None,None,"POLYGON ((-98.64 46.85, -98.64 69.47, -149.25 ..."
5,ABoVE: AirSWOT Ka-band Radar over Surface Wate...,ORNL_DAAC,C1604360562-ORNL_DAAC,ABoVE_AirSWOT_Radar_Data_1646,2017-07-08T00:00:00.000Z,2017-08-17T23:59:59.000Z,-149.83,46.85,-98.63,70.49,COMPLETE,"[EARTH SCIENCE, LAND SURFACE, TOPOGRAPHY, TERR...",https://doi.org/10.3334/ORNLDAAC/1646,https://daac.ornl.gov/ABOVE/guides/ABoVE_AirSW...,https://daac.ornl.gov/daacdata/above/ABoVE_Air...,None,None,"POLYGON ((-98.63 46.85, -98.63 70.489999999999..."
6,"ABoVE: Atmospheric Profiles of CO, CO2 and CH4...",ORNL_DAAC,C1610469826-ORNL_DAAC,ABoVE_Arctic_CAP_1658,2017-04-26T15:32:10.000Z,2017-11-05T23:35:40.000Z,-166.04,40.04,-104.11,71.29,COMPLETE,"[EARTH SCIENCE, ATMOSPHERE, ATMOSPHERIC CHEMIS...",https://doi.org/10.3334/ORNLDAAC/1658,https://daac.ornl.gov/ABOVE/guides/ABoVE_Arcti...,https://daac.ornl.gov/daacdata/above/ABoVE_Arc...,None,None,"POLYGON ((-104.11 40.04, -104.11 71.2900000000..."
7,ABoVE: AVHRR-Derived Forest Fire Burned Area-H...,ORNL_DAAC,C1456405338-ORNL_DAAC,AVHRR_Fire_Products_1545,1989-05-01T00:00:00.000Z,2000-10-31T23:59:59.000Z,-165.19,41.72,-53.17,69.01,COMPLETE,"[EARTH SCIENCE, BIOSPHERE, ECOLOGICAL DYNAMICS...",https://doi.org/10.3334/ORNLDAAC/1545,https://daac.ornl.gov/ABOVE/guides/AVHRR_Fire_...,https://daac.ornl.gov/daacdata/above/AVHRR_Fir...,None,None,"POLYGON ((-53.17 41.72, -53.17 69.010000000000..."
8,"ABoVE: Burn Severity, Fire Progression, and Fi...",ORNL_DAAC,C1422976988-ORNL_DAAC,Wildfires_NWT_Canada_1548,2015-05-20T00:00:00.000Z,2016-08-08T23:59:59.000Z,-135.54,59.93,-106.76,68.33,COMPLETE,"[EARTH SCIENCE, BIOSPHERE, ECOLOGICAL DYNAMICS...",https://doi.org/10.3334/ORNLDAAC/1548,https://daac.ornl.gov/ABOVE/guides/Wildfires_N...,http

#### Dump the table to binary file with C-optimized variant of the `pickle` library (both base py):
Python2: `CPickle`; Python3: `_pickle`

In [9]:
try:
    import _pickle as pickle
except ModuleNotFoundError:
    import pickle
    
with open(repo+'data/above_dataset_table.pkl', 'wb') as output:
    pickle.dump(dataset_table, output, -1)

#### Make sure it will open

In [10]:
with open(repo+'data/above_dataset_table.pkl', 'rb') as input:
    dataset_table = pickle.load(input)
dataset_table

,title,archive,conceptid,short_name,start_time,end_time,minlon,minlat,maxlon,maxlat,progress,science_keywords,url_landingpage,url_documentation,url_datapool,url_sdat,url_thredds,bounds_shapely
0,A Concise Experiment Plan for the Arctic-Borea...,ORNL_DAAC,C1536961538-ORNL_DAAC,ABoVE_Concise_Experiment_Plan_1617,2014-01-01T00:00:00.000Z,2021-12-31T23:59:59.000Z,-176.12,39.42,-66.92,81.61,COMPLETE,"[EARTH SCIENCE, ATMOSPHERE, ATMOSPHERIC CHEMIS...",https://doi.org/10.3334/ORNLDAAC/1617,https://daac.ornl.gov/ABOVE/guides/ABoVE_Conci...,https://daac.ornl.gov/daacdata/above/ABoVE_Con...,None,None,"POLYGON ((-66.92 39.42, -66.92 81.61, -176.12 ..."
1,ABoVE LVIS L1B Geolocated Return Energy Wavefo...,NSIDC_ECS,C1513105920-NSIDC_ECS,ABLVIS1B,2017-06-29T00:00:00.000Z,2017-07-17T23:59:59.999Z,-158.00,48.00,-104.00,72.00,PLANNED,"[EARTH SCIENCE, SPECTRAL/ENGINEERING, INFRARED...",https://doi.org/10.5067/UMRAWS57QAFU,None,https://search.earthdata.nasa.gov/search?q=ABL...,None,None,"POLYGON ((-104 48, -104 72, -158 72, -158 48, ..."
2,ABoVE LVIS L2 Geolocated Surface Elevation Pro...,NSIDC_ECS,C1513105984-NSIDC_ECS,ABLVIS2,2017-06-29T00:00:00.000Z,2017-07-17T23:59:59.999Z,-158.00,48.00,-104.00,72.00,PLANNED,"[EARTH SCIENCE, LAND SURFACE, TOPOGRAPHY, TERR...",https://doi.org/10.5067/IA5WAX7K3YGY,None,https://search.earthdata.nasa.gov/search?q=ABL...,None,None,"POLYGON ((-104 48, -104 72, -158 72, -158 48, ..."
3,ABoVE: Active Layer and Soil Moisture Properti...,ORNL_DAAC,C1608833566-ORNL_DAAC,ABoVE_PBand_SAR_1657,2014-08-16T00:00:00.000Z,2017-10-10T23:59:59.000Z,-167.94,64.71,-150.25,70.88,COMPLETE,"[EARTH SCIENCE, LAND SURFACE, FROZEN GROUND, A...",https://doi.org/10.3334/ORNLDAAC/1657,https://daac.ornl.gov/ABOVE/guides/ABoVE_PBand...,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,None,None,"POLYGON ((-150.25 64.70999999999999, -150.25 7..."
4,ABoVE: AirSWOT Color-Infrared Imagery Over Ala...,ORNL_DAAC,C1574863711-ORNL_DAAC,ABoVE_Open_Water_Map_1643,2017-07-09T00:00:00.000Z,2017-08-17T23:59:59.000Z,-149.25,46.85,-98.64,69.47,COMPLETE,"[EARTH SCIENCE, SPECTRAL/ENGINEERING, INFRARED...",https://doi.org/10.3334/ORNLDAAC/1643,https://daac.ornl.gov/ABOVE/guides/ABoVE_Open_...,https://daac.ornl.gov/daacdata/above/ABoVE_Ope...,None,None,"POLYGON ((-98.64 46.85, -98.64 69.47, -149.25 ..."
5,ABoVE: AirSWOT Ka-band Radar over Surface Wate...,ORNL_DAAC,C1604360562-ORNL_DAAC,ABoVE_AirSWOT_Radar_Data_1646,2017-07-08T00:00:00.000Z,2017-08-17T23:59:59.000Z,-149.83,46.85,-98.63,70.49,COMPLETE,"[EARTH SCIENCE, LAND SURFACE, TOPOGRAPHY, TERR...",https://doi.org/10.3334/ORNLDAAC/1646,https://daac.ornl.gov/ABOVE/guides/ABoVE_AirSW...,https://daac.ornl.gov/daacdata/above/ABoVE_Air...,None,None,"POLYGON ((-98.63 46.85, -98.63 70.489999999999..."
6,"ABoVE: Atmospheric Profiles of CO, CO2 and CH4...",ORNL_DAAC,C1610469826-ORNL_DAAC,ABoVE_Arctic_CAP_1658,2017-04-26T15:32:10.000Z,2017-11-05T23:35:40.000Z,-166.04,40.04,-104.11,71.29,COMPLETE,"[EARTH SCIENCE, ATMOSPHERE, ATMOSPHERIC CHEMIS...",https://doi.org/10.3334/ORNLDAAC/1658,https://daac.ornl.gov/ABOVE/guides/ABoVE_Arcti...,https://daac.ornl.gov/daacdata/above/ABoVE_Arc...,None,None,"POLYGON ((-104.11 40.04, -104.11 71.2900000000..."
7,ABoVE: AVHRR-Derived Forest Fire Burned Area-H...,ORNL_DAAC,C1456405338-ORNL_DAAC,AVHRR_Fire_Products_1545,1989-05-01T00:00:00.000Z,2000-10-31T23:59:59.000Z,-165.19,41.72,-53.17,69.01,COMPLETE,"[EARTH SCIENCE, BIOSPHERE, ECOLOGICAL DYNAMICS...",https://doi.org/10.3334/ORNLDAAC/1545,https://daac.ornl.gov/ABOVE/guides/AVHRR_Fire_...,https://daac.ornl.gov/daacdata/above/AVHRR_Fir...,None,None,"POLYGON ((-53.17 41.72, -53.17 69.010000000000..."
8,"ABoVE: Burn Severity, Fire Progression, and Fi...",ORNL_DAAC,C1422976988-ORNL_DAAC,Wildfires_NWT_Canada_1548,2015-05-20T00:00:00.000Z,2016-08-08T23:59:59.000Z,-135.54,59.93,-106.76,68.33,COMPLETE,"[EARTH SCIENCE, BIOSPHERE, ECOLOGICAL DYNAMICS...",https://doi.org/10.3334/ORNLDAAC/1548,https://daac.ornl.gov/ABOVE/guides/Wildfires_N...,http

### Get dataset locator table

In [11]:
dataset_locator_table = dataset_table[[
    "title",
    "conceptid",
    "short_name",
    "bounds_shapely",
    "start_time",
    "end_time",
    "minlon",
    "maxlon",
    "minlat",
    "maxlat",
    "url_sdat",
    "url_thredds",
    "science_keywords"
]]

dataset_locator_table

,title,conceptid,short_name,bounds_shapely,start_time,end_time,minlon,maxlon,minlat,maxlat,url_sdat,url_thredds,science_keywords
0,A Concise Experiment Plan for the Arctic-Borea...,C1536961538-ORNL_DAAC,ABoVE_Concise_Experiment_Plan_1617,"POLYGON ((-66.92 39.42, -66.92 81.61, -176.12 ...",2014-01-01T00:00:00.000Z,2021-12-31T23:59:59.000Z,-176.12,-66.92,39.42,81.61,None,None,"[EARTH SCIENCE, ATMOSPHERE, ATMOSPHERIC CHEMIS..."
1,ABoVE LVIS L1B Geolocated Return Energy Wavefo...,C1513105920-NSIDC_ECS,ABLVIS1B,"POLYGON ((-104 48, -104 72, -158 72, -158 48, ...",2017-06-29T00:00:00.000Z,2017-07-17T23:59:59.999Z,-158.00,-104.00,48.00,72.00,None,None,"[EARTH SCIENCE, SPECTRAL/ENGINEERING, INFRARED..."
2,ABoVE LVIS L2 Geolocated Surface Elevation Pro...,C1513105984-NSIDC_ECS,ABLVIS2,"POLYGON ((-104 48, -104 72, -158 72, -158 48, ...",2017-06-29T00:00:00.000Z,2017-07-17T23:59:59.999Z,-158.00,-104.00,48.00,72.00,None,None,"[EARTH SCIENCE, LAND SURFACE, TOPOGRAPHY, TERR..."
3,ABoVE: Active Layer and Soil Moisture Properti...,C1608833566-ORNL_DAAC,ABoVE_PBand_SAR_1657,"POLYGON ((-150.25 64.70999999999999, -150.25 7...",2014-08-16T00:00:00.000Z,2017-10-10T23:59:59.000Z,-167.94,-150.25,64.71,70.88,None,None,"[EARTH SCIENCE, LAND SURFACE, FROZEN GROUND, A..."
4,ABoVE: AirSWOT Color-Infrared Imagery Over Ala...,C1574863711-ORNL_DAAC,ABoVE_Open_Water_Map_1643,"POLYGON ((-98.64 46.85, -98.64 69.47, -149.25 ...",2017-07-09T00:00:00.000Z,2017-08-17T23:59:59.000Z,-149.25,-98.64,46.85,69.47,None,None,"[EARTH SCIENCE, SPECTRAL/ENGINEERING, INFRARED..."
5,ABoVE: AirSWOT Ka-band Radar over Surface Wate...,C1604360562-ORNL_DAAC,ABoVE_AirSWOT_Radar_Data_1646,"POLYGON ((-98.63 46.85, -98.63 70.489999999999...",2017-07-08T00:00:00.000Z,2017-08-17T23:59:59.000Z,-149.83,-98.63,46.85,70.49,None,None,"[EARTH SCIENCE, LAND SURFACE, TOPOGRAPHY, TERR..."
6,"ABoVE: Atmospheric Profiles of CO, CO2 and CH4...",C1610469826-ORNL_DAAC,ABoVE_Arctic_CAP_1658,"POLYGON ((-104.11 40.04, -104.11 71.2900000000...",2017-04-26T15:32:10.000Z,2017-11-05T23:35:40.000Z,-166.04,-104.11,40.04,71.29,None,None,"[EARTH SCIENCE, ATMOSPHERE, ATMOSPHERIC CHEMIS..."
7,ABoVE: AVHRR-Derived Forest Fire Burned Area-H...,C1456405338-ORNL_DAAC,AVHRR_Fire_Products_1545,"POLYGON ((-53.17 41.72, -53.17 69.010000000000...",1989-05-01T00:00:00.000Z,2000-10-31T23:59:59.000Z,-165.19,-53.17,41.72,69.01,None,None,"[EARTH SCIENCE, BIOSPHERE, ECOLOGICAL DYNAMICS..."
8,"ABoVE: Burn Severity, Fire Progression, and Fi...",C1422976988-ORNL_DAAC,Wildfires_NWT_Canada_1548,"POLYGON ((-106.76 59.93, -106.76 68.33, -135.5...",2015-05-20T00:00:00.000Z,2016-08-08T23:59:59.000Z,-135.54,-106.76,59.93,68.33,None,None,"[EARTH SCIENCE, BIOSPHERE, ECOLOGICAL DYNAMICS..."
9,"ABoVE: Burn Severity, Fire Progression, Landco...",C1248454748-ORNL_DAAC,Wildfires_2014_NWT_Canada_1307,"POLYGON ((-110.68 60.33, -110.68 64.25, -121.6...",1997-07-07T00:00:00.000Z,2015-07-15T23:59:59.000Z,-121.60,-110.68,60.33,64.25,https://webmap.ornl.gov/wcsdown/dataset.jsp?ds...,None,"[EARTH SCIENCE, BIOSPHERE, ECOLOGICAL DYNAMICS..."


## CMR Granules

In [12]:
def get_granules_table(cmr_granules_response_dictionary):
    """ """
    
    granule_rows = []

    for granule in cmr_granules_response_dictionary["items"]:
        
        umm = granule["umm"]
        meta = granule["meta"]
        archive = meta["provider-id"]
        conceptid = meta["concept-id"]
        
        try:
            granuleid = meta["native-id"]
        except:
            granuleid = None
            
        if (len(umm["Projects"])==1)&(granuleid is not None):

            try:
                collection_short_name = umm["CollectionReference"]["ShortName"]
            except:
                collection_short_name = None
            
            try:
                granule_size = umm["DataGranule"]["ArchiveAndDistributionInformation"]["Size"]
            except:
                granule_size = None
            
            try:
                granule_params = get_granule_parameters(umm)
            except:
                granule_params = None
            
            range_time = umm["TemporalExtent"]["RangeDateTime"]
            start_time = range_time["BeginningDateTime"]
            end_time = range_time["EndingDateTime"]
                        
            url_datapool = get_urls(umm, "granules")
            
            try:
                bounds = get_bounds(umm)
                bounds_shapely = get_bounds_shapely(bounds)
                minlon = bounds[0]
                maxlon = bounds[1]
                minlat = bounds[3]
                maxlat = bounds[2]

                granule_rows.append((
                    archive,
                    collection_short_name,
                    conceptid,
                    granuleid,
                    granule_size,
                    granule_params,
                    start_time,
                    end_time,
                    minlon,
                    maxlon,
                    minlat,
                    maxlat,
                    url_datapool,
                    bounds_shapely
                ))
            except:
                pass


    return(pd.DataFrame(granule_rows, columns=[
        "archive",
        "collection_short_name",
        "conceptid",
        "granuleid",
        "granule_size",
        "granule_params",
        "start_time",
        "end_time",
        "minlon",
        "maxlon",
        "minlat",
        "maxlat",
        "url_datapool",
        "bounds_shapely"
    ]))

#### Test it

In [13]:
granule_table = get_granules_table(above_dataset_granules)

granule_table

,archive,collection_short_name,conceptid,granuleid,granule_size,granule_params,start_time,end_time,minlon,maxlon,minlat,maxlat,url_datapool,bounds_shapely
0,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749265-ORNL_DAAC,Maps_AGB_North_Slope_AK.shrub_agb_p50.tif,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."
1,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749266-ORNL_DAAC,Maps_AGB_North_Slope_AK.shrub_dominance_of_agb...,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."
2,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749267-ORNL_DAAC,Maps_AGB_North_Slope_AK.plant_agb_p2_5.tif,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."
3,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749268-ORNL_DAAC,Maps_AGB_North_Slope_AK.shrub_dominance_of_agb...,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."
4,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749270-ORNL_DAAC,Maps_AGB_North_Slope_AK.shrub_agb_p2_5.tif,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."
5,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749271-ORNL_DAAC,Maps_AGB_North_Slope_AK.shrub_agb_p97_5.tif,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."
6,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749272-ORNL_DAAC,Maps_AGB_North_Slope_AK.shrub_dominance_of_agb...,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."
7,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749273-ORNL_DAAC,Maps_AGB_North_Slope_AK.plant_agb_p97_5.tif,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."
8,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749274-ORNL_DAAC,Maps_AGB_North_Slope_AK.tundra_biomass_harvest...,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-164.634,64.7226,76.226,-111.551,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-111.551 64.7226, -111.551 76.226, -..."
9,ORNL_DAAC,Maps_AGB_North_Slope_AK_1565,G1513749275-ORNL_DAAC,Maps_AGB_North_Slope_AK.plant_agb_p50.tif,None,"[VEGETATION INDEX, BIOMASS, VEGETATION COVER]",2007-06-01T00:00:00.000Z,2016-08-31T23:59:00.000Z,-168.576,66.9150,70.915,-139.932,https://daac.ornl.gov/daacdata/above/Maps_AGB_...,"POLYGON ((-139.932 66.91500000000001, -139.932..."


#### Make a granules table for each ABoVE dataset and concatenate:

In [14]:
pdict = dict(collection_concept_id="C1248454748-ORNL_DAAC", page_size="2000")
gdict = cmr_search("granules.umm_json_v1_4?", pdict)
gdict

{'hits': 12,
 'took': 66,
 'items': [{'meta': {'concept-type': 'granule',
    'concept-id': 'G1422951257-ORNL_DAAC',
    'revision-id': 1,
    'native-id': 'Wildfires_2014_NWT_Canada.NWT_Wildfire_sites_ss68.tif',
    'provider-id': 'ORNL_DAAC',
    'format': 'application/echo10+xml',
    'revision-date': '2017-09-28T19:04:33Z'},
   'umm': {'TemporalExtent': {'RangeDateTime': {'BeginningDateTime': '1997-07-07T00:00:00.000Z',
      'EndingDateTime': '2013-08-04T00:00:00.000Z'}},
    'GranuleUR': 'Wildfires_2014_NWT_Canada.NWT_Wildfire_sites_ss68.tif',
    'MeasuredParameters': [{'ParameterName': 'FIRE OCCURRENCE'},
     {'ParameterName': 'DOMINANT SPECIES'},
     {'ParameterName': 'SOIL MOISTURE/WATER CONTENT'},
     {'ParameterName': 'BIOMASS'},
     {'ParameterName': 'LAND USE/LAND COVER CLASSIFICATION'},
     {'ParameterName': 'PLANT CHARACTERISTICS'},
     {'ParameterName': 'SOIL TEMPERATURE'},
     {'ParameterName': 'VEGETATION COVER'}],
    'SpatialExtent': {'HorizontalSpatialDomai

In [15]:
granules_table_list = []

for index, row in dataset_table.iterrows():
    
    conceptid = row["conceptid"]
    params_dict = dict(collection_concept_id=conceptid, page_size="2000")
    granules_dict = cmr_search("granules.umm_json_v1_4?", params_dict)
    granule_table = get_granules_table(granules_dict)
    
    granules_table_list.append(granule_table)

above_granules_table = pd.concat(granules_table_list)
above_granules_table.reset_index(drop=True, inplace=True)

above_granules_table

,archive,collection_short_name,conceptid,granuleid,granule_size,granule_params,start_time,end_time,minlon,maxlon,minlat,maxlat,url_datapool,bounds_shapely
0,ORNL_DAAC,ABoVE_Concise_Experiment_Plan_1617,G1546816353-ORNL_DAAC,ABoVE_Concise_Experiment_Plan.Concise_Experime...,None,"[ALPINE/TUNDRA, FORESTS, CARBON DIOXIDE, METHA...",2014-01-01T00:00:00.000Z,2021-12-31T00:00:00.000Z,-176.125,39.4150,81.6086,-66.9178,https://daac.ornl.gov/daacdata/above/ABoVE_Con...,"POLYGON ((-66.9178 39.415, -66.9178 81.6086, -..."
1,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836047-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_ivotu...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-158.958,68.0962,68.3564,-158.3650,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-158.365 68.0962, -158.365 68.356399..."
2,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836048-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_kouga...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-165.287,65.4095,65.6772,-164.7960,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-164.796 65.40949999999999, -164.796..."
3,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836056-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_dhors...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-151.683,69.5411,69.7892,-151.0000,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-151 69.5411, -151 69.78919999999999..."
4,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836057-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_koyuk...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-164.602,64.7127,64.9793,-164.1180,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-164.118 64.7127, -164.118 64.979299..."
5,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836061-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_counc...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-166.678,64.7712,65.0404,-166.2090,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-166.209 64.77119999999999, -166.209..."
6,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836064-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_atqas...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-160.908,69.7700,70.0348,-160.2860,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-160.286 69.77, -160.286 70.0348, -1..."
7,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836065-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_barro...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-158.916,70.6124,70.8750,-158.2550,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-158.255 70.61239999999999, -158.255..."
8,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836067-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_amble...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-162.859,66.0011,66.2659,-162.3390,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-162.339 66.00109999999999, -162.339..."
9,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836046-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_kouga...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2015-08-29T00:00:00.000Z,2015-10-01T00:00:00.000Z,-163.582,65.4359,65.7015,-163.0780,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-163.078 65.4359, -163.078 65.7015, ..."


#### Pickle to another binary file

In [16]:
with open(repo+'data/above_granules_table.pkl', 'wb') as output:
    pickle.dump(above_granules_table, output, -1)

#### Make sure it opens

In [17]:
with open(repo+'data/above_granules_table.pkl', 'rb') as input:
    above_granules_tableabove_granules_table = pickle.load(input)
    
above_granules_table

,archive,collection_short_name,conceptid,granuleid,granule_size,granule_params,start_time,end_time,minlon,maxlon,minlat,maxlat,url_datapool,bounds_shapely
0,ORNL_DAAC,ABoVE_Concise_Experiment_Plan_1617,G1546816353-ORNL_DAAC,ABoVE_Concise_Experiment_Plan.Concise_Experime...,None,"[ALPINE/TUNDRA, FORESTS, CARBON DIOXIDE, METHA...",2014-01-01T00:00:00.000Z,2021-12-31T00:00:00.000Z,-176.125,39.4150,81.6086,-66.9178,https://daac.ornl.gov/daacdata/above/ABoVE_Con...,"POLYGON ((-66.9178 39.415, -66.9178 81.6086, -..."
1,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836047-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_ivotu...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-158.958,68.0962,68.3564,-158.3650,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-158.365 68.0962, -158.365 68.356399..."
2,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836048-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_kouga...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-165.287,65.4095,65.6772,-164.7960,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-164.796 65.40949999999999, -164.796..."
3,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836056-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_dhors...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-151.683,69.5411,69.7892,-151.0000,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-151 69.5411, -151 69.78919999999999..."
4,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836057-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_koyuk...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-164.602,64.7127,64.9793,-164.1180,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-164.118 64.7127, -164.118 64.979299..."
5,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836061-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_counc...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-166.678,64.7712,65.0404,-166.2090,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-166.209 64.77119999999999, -166.209..."
6,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836064-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_atqas...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-160.908,69.7700,70.0348,-160.2860,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-160.286 69.77, -160.286 70.0348, -1..."
7,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836065-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_barro...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-158.916,70.6124,70.8750,-158.2550,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-158.255 70.61239999999999, -158.255..."
8,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836067-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_amble...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-162.859,66.0011,66.2659,-162.3390,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-162.339 66.00109999999999, -162.339..."
9,ORNL_DAAC,ABoVE_PBand_SAR_1657,G1608836046-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_kouga...,None,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",2015-08-29T00:00:00.000Z,2015-10-01T00:00:00.000Z,-163.582,65.4359,65.7015,-163.0780,https://daac.ornl.gov/daacdata/above/ABoVE_PBa...,"POLYGON ((-163.078 65.4359, -163.078 65.7015, ..."


# App-specific items
#### Import app packages and load app data from file(s):

In [18]:
gridf = "../data/ABoVE_240m_30m_5m_grid_tiles/ABoVE_240m_30m_5m_grid_tiles.json"
domainf = "../data/ABoVE_Study_Domain/ABoVE_Study_Domain.json"

with open(gridf, 'r') as file:
    above_grid = json.load(file)

with open(domainf, 'r') as file:
    above_domain = json.load(file)

#### Make a smaller version of the table that is referenced after interactions

In [19]:
granule_locator_table = above_granules_table[[
    "collection_short_name",
    "conceptid",
    "granuleid",
    "bounds_shapely",
    "start_time",
    "end_time",
    "minlon",
    "maxlon",
    "minlat",
    "maxlat",
    "granule_params",
    "url_datapool"
]]

granule_locator_table

,collection_short_name,conceptid,granuleid,bounds_shapely,start_time,end_time,minlon,maxlon,minlat,maxlat,granule_params,url_datapool
0,ABoVE_Concise_Experiment_Plan_1617,G1546816353-ORNL_DAAC,ABoVE_Concise_Experiment_Plan.Concise_Experime...,"POLYGON ((-66.9178 39.415, -66.9178 81.6086, -...",2014-01-01T00:00:00.000Z,2021-12-31T00:00:00.000Z,-176.125,39.4150,81.6086,-66.9178,"[ALPINE/TUNDRA, FORESTS, CARBON DIOXIDE, METHA...",https://daac.ornl.gov/daacdata/above/ABoVE_Con...
1,ABoVE_PBand_SAR_1657,G1608836047-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_ivotu...,"POLYGON ((-158.365 68.0962, -158.365 68.356399...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-158.958,68.0962,68.3564,-158.3650,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...
2,ABoVE_PBand_SAR_1657,G1608836048-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_kouga...,"POLYGON ((-164.796 65.40949999999999, -164.796...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-165.287,65.4095,65.6772,-164.7960,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...
3,ABoVE_PBand_SAR_1657,G1608836056-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_dhors...,"POLYGON ((-151 69.5411, -151 69.78919999999999...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-151.683,69.5411,69.7892,-151.0000,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...
4,ABoVE_PBand_SAR_1657,G1608836057-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_koyuk...,"POLYGON ((-164.118 64.7127, -164.118 64.979299...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-164.602,64.7127,64.9793,-164.1180,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...
5,ABoVE_PBand_SAR_1657,G1608836061-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_counc...,"POLYGON ((-166.209 64.77119999999999, -166.209...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-166.678,64.7712,65.0404,-166.2090,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...
6,ABoVE_PBand_SAR_1657,G1608836064-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_atqas...,"POLYGON ((-160.286 69.77, -160.286 70.0348, -1...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-160.908,69.7700,70.0348,-160.2860,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...
7,ABoVE_PBand_SAR_1657,G1608836065-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_barro...,"POLYGON ((-158.255 70.61239999999999, -158.255...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-158.916,70.6124,70.8750,-158.2550,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...
8,ABoVE_PBand_SAR_1657,G1608836067-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_amble...,"POLYGON ((-162.339 66.00109999999999, -162.339...",2014-08-16T00:00:00.000Z,2014-10-09T00:00:00.000Z,-162.859,66.0011,66.2659,-162.3390,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...
9,ABoVE_PBand_SAR_1657,G1608836046-ORNL_DAAC,ABoVE_PBand_SAR.PolSAR_active_layer_prop_kouga...,"POLYGON ((-163.078 65.4359, -163.078 65.7015, ...",2015-08-29T00:00:00.000Z,2015-10-01T00:00:00.000Z,-163.582,65.4359,65.7015,-163.0780,"[SOIL MOISTURE/WATER CONTENT, SURFACE ROUGHNES...",https://daac.ornl.gov/daacdata/above/ABoVE_PBa...


## ABoVE Grid

#### Make table describing the enabled ABoVE tiles:

In [20]:
enabled_levels = ["A", "B"] # "C"

rows = []

for feature in above_grid["features"]:
    
    geometry = feature["geometry"]
    properties = feature["properties"]
    
    grid_level = properties["grid_level"]
    
    if grid_level in enabled_levels:
        
        grid_id = properties["grid_id"]
        spatial_re = properties["spatial_re"]

        ah = properties["ah"]
        av = properties["av"]
        bh = properties["bh"]
        bv = properties["bv"]
        ch = properties["ch"]
        cv = properties["cv"]

        geom_grid = shape(geometry)

        # DATASETS SELECTION
        
        dataset_locator_indices = []
        dataset_conceptid = []

        for index, row in dataset_locator_table.iterrows():
            geom_dataset = row["bounds_shapely"]

            if geom_dataset.intersects(geom_grid):
                dataset_locator_indices.append(index)
                dataset_conceptid.append(row["conceptid"])

        dataset_count = len(dataset_locator_indices)
        
        # GRANULES SELECTION
        
        granule_locator_indices = []

        for index, row in granule_locator_table.iterrows():
            geom_dataset = row["bounds_shapely"]

            if geom_dataset.intersects(geom_grid):
                granule_locator_indices.append(index)

        granule_count = len(granule_locator_indices)

        rows.append((
            geometry,
            geom_grid,
            properties,
            grid_level,
            grid_id,
            spatial_re,
            ah,
            av,
            bh,
            bv,
            ch,
            cv,
            dataset_count,
            dataset_locator_indices,
            dataset_conceptid,
            granule_count,
            granule_locator_indices
        ))

above_grid_table = pd.DataFrame(rows, columns=[
    "geometry",
    "bounds_shapely",
    "properties",
    "grid_level",
    "grid_id",
    "spatial_re",
    "ah",
    "av",
    "bh",
    "bv",
    "ch",
    "cv",
    "dataset_count",
    "dataset_locator_ix",
    "dataset_conceptid",
    "granule_count",
    "granule_locator_ix",
])
    
above_grid_table

,geometry,bounds_shapely,properties,grid_level,grid_id,spatial_re,ah,av,bh,bv,ch,cv,dataset_count,dataset_locator_ix,dataset_conceptid,granule_count,granule_locator_ix
0,"{'type': 'Polygon', 'coordinates': [[[-177.469...",POLYGON ((-177.4690049761034 60.55402887358052...,"{'grid_level': 'A', 'grid_id': 'Ah000v000', 's...",A,Ah000v000,240 meter (coarse),0,0,0,0,0,0,38,"[0, 1, 2, 3, 4, 5, 6, 7, 14, 15, 16, 18, 19, 2...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1711,"[0, 2, 4, 5, 8, 9, 11, 13, 14, 17, 24, 25, 26,..."
1,"{'type': 'Polygon', 'coordinates': [[[-148.444...",POLYGON ((-148.4442125539481 49.86615911532742...,"{'grid_level': 'A', 'grid_id': 'Ah000v002', 's...",A,Ah000v002,240 meter (coarse),0,2,0,0,0,0,22,"[0, 1, 2, 4, 5, 6, 7, 18, 19, 21, 22, 23, 25, ...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1179,"[0, 43, 47, 48, 50, 58, 365, 1910, 1911, 1912,..."
2,"{'type': 'Polygon', 'coordinates': [[[-135.930...",POLYGON ((-135.9305914447084 56.01514057706573...,"{'grid_level': 'A', 'grid_id': 'Ah001v002', 's...",A,Ah001v002,240 meter (coarse),1,2,0,0,0,0,28,"[0, 1, 2, 4, 5, 6, 7, 8, 9, 11, 14, 18, 19, 21...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1721,"[0, 43, 47, 48, 50, 58, 365, 1910, 1911, 1912,..."
3,"{'type': 'Polygon', 'coordinates': [[[-119.290...",POLYGON ((-119.2902530971568 60.38576901856309...,"{'grid_level': 'A', 'grid_id': 'Ah002v002', 's...",A,Ah002v002,240 meter (coarse),2,2,0,0,0,0,28,"[0, 1, 2, 4, 5, 6, 7, 8, 9, 11, 14, 18, 19, 21...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",2689,"[0, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40..."
4,"{'type': 'Polygon', 'coordinates': [[[-99.1297...",POLYGON ((-99.12976833300675 62.27196651365501...,"{'grid_level': 'A', 'grid_id': 'Ah003v002', 's...",A,Ah003v002,240 meter (coarse),3,2,0,0,0,0,17,"[0, 4, 5, 7, 14, 19, 23, 25, 26, 27, 28, 32, 3...","[C1536961538-ORNL_DAAC, C1574863711-ORNL_DAAC,...",1242,"[0, 48, 365, 1910, 1911, 1912, 1913, 1914, 191..."
5,"{'type': 'Polygon', 'coordinates': [[[-78.4070...",POLYGON ((-78.40702219488523 61.23415405017053...,"{'grid_level': 'A', 'grid_id': 'Ah004v002', 's...",A,Ah004v002,240 meter (coarse),4,2,0,0,0,0,13,"[0, 7, 14, 19, 26, 27, 28, 32, 33, 36, 60, 71,...","[C1536961538-ORNL_DAAC, C1456405338-ORNL_DAAC,...",1205,"[0, 1910, 1911, 1912, 1913, 1914, 1915, 1916, ..."
6,"{'type': 'Polygon', 'coordinates': [[[-60.5565...","POLYGON ((-60.5565988622706 57.52655951000388,...","{'grid_level': 'A', 'grid_id': 'Ah005v002', 's...",A,Ah005v002,240 meter (coarse),5,2,0,0,0,0,12,"[0, 7, 19, 26, 27, 28, 32, 33, 36, 60, 71, 73]","[C1536961538-ORNL_DAAC, C1456405338-ORNL_DAAC,...",1126,"[0, 1910, 1911, 1912, 1913, 1914, 1915, 1916, ..."
7,"{'type': 'Polygon', 'coordinates': [[[-139.379...","POLYGON ((-139.379266966226 42.42459121741177,...","{'grid_level': 'A', 'grid_id': 'Ah000v003', 's...",A,Ah000v003,240 meter (coarse),0,3,0,0,0,0,13,"[0, 4, 5, 6, 7, 19, 26, 27, 32, 33, 36, 71, 73]","[C1536961538-ORNL_DAAC, C1574863711-ORNL_DAAC,...",1013,"[0, 48, 365, 1910, 1911, 1912, 1913, 1914, 191..."
8,"{'type': 'Polygon', 'coordinates': [[[-127.903...",POLYGON ((-127.9034040846061 47.67873631568072...,"{'grid_level': 'A', 'grid_id': 'Ah001v003', 's...",A,Ah001v003,240 meter (coarse),1,3,0,0,0,0,18,"[0, 1, 2, 4, 5, 6, 7, 19, 22, 26, 27, 28, 30, ...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1051,"[0, 48, 365, 1910, 1911, 1912, 1913, 1914, 191..."
9,"{'type': 'Polygon', 'coordinates': [[[-114.016...",POLYGON ((-114.0165009351113 51.22820222928021...,"{'grid_level': 'A', 'grid_id': 'Ah002v003', 's...",A,Ah002v003,240 meter (coarse),2,3,0,0,0,0,21,"[0, 1, 2, 4, 5, 6, 7, 19, 21, 22, 23, 25, 26, ...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1213,"[0, 48, 327, 330, 332, 342, 344, 352, 359, 365..."


#### Dump the above_grid_table to a file to read later:

In [21]:
with open(repo+"data/above_grid_table_ab.pkl", 'wb') as output:
    pickle.dump(above_grid_table, output, -1)

#### Make sure it reads correctly:

In [22]:
with open(repo+'data/above_grid_table_ab.pkl', 'rb') as input:
    above_grid_table = pickle.load(input)
    
above_grid_table

,geometry,bounds_shapely,properties,grid_level,grid_id,spatial_re,ah,av,bh,bv,ch,cv,dataset_count,dataset_locator_ix,dataset_conceptid,granule_count,granule_locator_ix
0,"{'type': 'Polygon', 'coordinates': [[[-177.469...",POLYGON ((-177.4690049761034 60.55402887358052...,"{'grid_level': 'A', 'grid_id': 'Ah000v000', 's...",A,Ah000v000,240 meter (coarse),0,0,0,0,0,0,38,"[0, 1, 2, 3, 4, 5, 6, 7, 14, 15, 16, 18, 19, 2...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1711,"[0, 2, 4, 5, 8, 9, 11, 13, 14, 17, 24, 25, 26,..."
1,"{'type': 'Polygon', 'coordinates': [[[-148.444...",POLYGON ((-148.4442125539481 49.86615911532742...,"{'grid_level': 'A', 'grid_id': 'Ah000v002', 's...",A,Ah000v002,240 meter (coarse),0,2,0,0,0,0,22,"[0, 1, 2, 4, 5, 6, 7, 18, 19, 21, 22, 23, 25, ...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1179,"[0, 43, 47, 48, 50, 58, 365, 1910, 1911, 1912,..."
2,"{'type': 'Polygon', 'coordinates': [[[-135.930...",POLYGON ((-135.9305914447084 56.01514057706573...,"{'grid_level': 'A', 'grid_id': 'Ah001v002', 's...",A,Ah001v002,240 meter (coarse),1,2,0,0,0,0,28,"[0, 1, 2, 4, 5, 6, 7, 8, 9, 11, 14, 18, 19, 21...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1721,"[0, 43, 47, 48, 50, 58, 365, 1910, 1911, 1912,..."
3,"{'type': 'Polygon', 'coordinates': [[[-119.290...",POLYGON ((-119.2902530971568 60.38576901856309...,"{'grid_level': 'A', 'grid_id': 'Ah002v002', 's...",A,Ah002v002,240 meter (coarse),2,2,0,0,0,0,28,"[0, 1, 2, 4, 5, 6, 7, 8, 9, 11, 14, 18, 19, 21...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",2689,"[0, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40..."
4,"{'type': 'Polygon', 'coordinates': [[[-99.1297...",POLYGON ((-99.12976833300675 62.27196651365501...,"{'grid_level': 'A', 'grid_id': 'Ah003v002', 's...",A,Ah003v002,240 meter (coarse),3,2,0,0,0,0,17,"[0, 4, 5, 7, 14, 19, 23, 25, 26, 27, 28, 32, 3...","[C1536961538-ORNL_DAAC, C1574863711-ORNL_DAAC,...",1242,"[0, 48, 365, 1910, 1911, 1912, 1913, 1914, 191..."
5,"{'type': 'Polygon', 'coordinates': [[[-78.4070...",POLYGON ((-78.40702219488523 61.23415405017053...,"{'grid_level': 'A', 'grid_id': 'Ah004v002', 's...",A,Ah004v002,240 meter (coarse),4,2,0,0,0,0,13,"[0, 7, 14, 19, 26, 27, 28, 32, 33, 36, 60, 71,...","[C1536961538-ORNL_DAAC, C1456405338-ORNL_DAAC,...",1205,"[0, 1910, 1911, 1912, 1913, 1914, 1915, 1916, ..."
6,"{'type': 'Polygon', 'coordinates': [[[-60.5565...","POLYGON ((-60.5565988622706 57.52655951000388,...","{'grid_level': 'A', 'grid_id': 'Ah005v002', 's...",A,Ah005v002,240 meter (coarse),5,2,0,0,0,0,12,"[0, 7, 19, 26, 27, 28, 32, 33, 36, 60, 71, 73]","[C1536961538-ORNL_DAAC, C1456405338-ORNL_DAAC,...",1126,"[0, 1910, 1911, 1912, 1913, 1914, 1915, 1916, ..."
7,"{'type': 'Polygon', 'coordinates': [[[-139.379...","POLYGON ((-139.379266966226 42.42459121741177,...","{'grid_level': 'A', 'grid_id': 'Ah000v003', 's...",A,Ah000v003,240 meter (coarse),0,3,0,0,0,0,13,"[0, 4, 5, 6, 7, 19, 26, 27, 32, 33, 36, 71, 73]","[C1536961538-ORNL_DAAC, C1574863711-ORNL_DAAC,...",1013,"[0, 48, 365, 1910, 1911, 1912, 1913, 1914, 191..."
8,"{'type': 'Polygon', 'coordinates': [[[-127.903...",POLYGON ((-127.9034040846061 47.67873631568072...,"{'grid_level': 'A', 'grid_id': 'Ah001v003', 's...",A,Ah001v003,240 meter (coarse),1,3,0,0,0,0,18,"[0, 1, 2, 4, 5, 6, 7, 19, 22, 26, 27, 28, 30, ...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1051,"[0, 48, 365, 1910, 1911, 1912, 1913, 1914, 191..."
9,"{'type': 'Polygon', 'coordinates': [[[-114.016...",POLYGON ((-114.0165009351113 51.22820222928021...,"{'grid_level': 'A', 'grid_id': 'Ah002v003', 's...",A,Ah002v003,240 meter (coarse),2,3,0,0,0,0,21,"[0, 1, 2, 4, 5, 6, 7, 19, 21, 22, 23, 25, 26, ...","[C1536961538-ORNL_DAAC, C1513105920-NSIDC_ECS,...",1213,"[0, 48, 327, 330, 332, 342, 344, 352, 359, 365..."
